In [151]:
include("compress.jl")

rlcs_unzip (generic function with 1 method)

In [ ]:

function lcs_zip(str, refer) # `str` and `refer` must not contain any uppercase letters
    rg1, rg2 = lcs(refer, str)
    if length(rg1) > 2 || (length(rg1) == 2 && rg1[1] == 1)
        b1, e1 = first(rg1), last(rg1)
        b2, e2 = first(rg2), last(rg2)
        l1 = e1 - b1 + 1
        if b1 <= 26 && l1 <= 26
            bc = b1 == 1 ? "" : 'A' + (b1 - 1)
            lc = 'A' + (l1 - 1)
            str = string(str[1:b2-1], bc, lc, str[e2+1:end])
        end
    end
    str
end

function rlcs_unzip(Z, str)
    _b2 = findfirst(r"[A-Z]", str)
    if _b2 !== nothing
        _b2 = first(_b2)
        rk = str[_b2] != 'A' ? 1 : str[_b2 + 1] - '1' + 1
        if str[_b2] != 'A'
            b2 = _b2
            rk = 1
        else
            b2 = first(findnext(r"[A-Z]", str, _b2 + 2))
            rk = str[_b2 + 1] - '1' + 1
        end
        refer = Z.pool[(Z.counter - rk + 1) % Z.rank + 1]
        b1 = str[b2] - 'A' + 1
        e2 = b2 + 1
        if e2 > length(str) || !('A' <= str[e2] <= 'Z')
            e2 = b2
            l1 = b1
            b1 = 1
        else
            l1 = str[e2] - 'A' + 1
        end
        e1 = b1 + l1 - 1
        str = string(str[1:_b2-1], refer[b1:e1], str[e2+1:end])
    end
    Z.counter += 1
    Z.pool[Z.counter % Z.rank + 1] = str
    return str
end

function rlcs_unzip(Z, str)
    _b2 = findfirst(r"[A-Z]", str)
    if _b2 !== nothing
        _b2 = first(_b2)
        rk = str[_b2] != 'A' ? 1 : str[_b2 + 1] - '1' + 1
        if str[_b2] != 'A'
            b2 = _b2
            rk = 1
        else
            b2 = first(findfirst(r"[A-Z]", str, _b2 + 2))
            rk = str[_b2 + 1] - '1' + 1
        end
        refer = Z.pool[(Z.counter - rk + 1) % Z.rank + 1]
        b1 = str[b2] - 'A' + 1
        e2 = b2 + 1
        if e2 > length(str) || !('A' <= str[e2] <= 'Z')
            e2 = b2
            l1 = b1
            b1 = 1
        else
            l1 = str[e2] - 'A' + 1
        end
        e1 = b1 + l1 - 1
        str = string(str[1:_b2-1], refer[b1:e1], str[e2+1:end])
    end
    Z.counter += 1
    Z.pool[Z.counter % Z.rank + 1] = str
    return str
end
mutable struct RLCS
    rank::Int
    pool::Vector{String}
    counter::Int
end
function RLCS(rank::Int=9)
    return RLCS(rank, fill("", rank), 0)
end
function rlcs_zip(Z, str)
    best = 0, str, "", ""
    for rk in 1:min(Z.rank, Z.counter)
        refer = Z.pool[(Z.counter - rk + 1) % Z.rank + 1]
        # @show rk refer
        l, h, c, t = lcs_zip(str, refer)
        rk > 1 && (l -= 2)
        # @show l
        if l > best[1]
            s = rk == 1 ? "" : 'A' * ('1' + rk - 1)
            best = l, h, s * c, t
            # @show s
        end
        if length(str) - l < 2
            break
        end
    end
    Z.counter += 1
    zip_str = string(best[2], best[3], best[4])
    Z.pool[Z.counter % Z.rank + 1] = str
    return zip_str
end

function rlcs_unzip(Z, str)
    _b2 = findfirst(r"[A-Z]", str)
    if _b2 !== nothing
        _b2 = first(_b2)
        rk = str[_b2] != 'A' ? 1 : str[_b2 + 1] - '1' + 1
        if str[_b2] != 'A'
            b2 = _b2
            rk = 1
        else
            b2 = first(findnext(r"[A-Z]", str, _b2 + 2))
            rk = str[_b2 + 1] - '1' + 1
        end
        refer = Z.pool[(Z.counter - rk + 1) % Z.rank + 1]
        b1 = str[b2] - 'A' + 1
        e2 = b2 + 1
        if e2 > length(str) || !('A' <= str[e2] <= 'Z')
            e2 = b2
            l1 = b1
            b1 = 1
        else
            l1 = str[e2] - 'A' + 1
        end
        e1 = b1 + l1 - 1
        str = string(str[1:_b2-1], refer[b1:e1], str[e2+1:end])
    end
    Z.counter += 1
    Z.pool[Z.counter % Z.rank + 1] = str
    return str
end



In [168]:
1:36

35

In [171]:
Int('0')

48

In [170]:
Int('a')

97

In [213]:
ec(i) = i <= 11 ? ('0' + i - 2) : (i <= 37 ? 'a' + i - 12 : 'A' + i - 38)
dc(c) = c <= '9' ? (c - '0' + 2) : (c >= 'a' ? c - 'a' + 12 : c - 'A' + 38)


dc (generic function with 1 method)

In [214]:
ec.(1:63)|>print

['/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

In [216]:
dc.(ec.(1:63)) == 1:63

true

In [199]:
Int('A')

65